<a href="https://colab.research.google.com/github/Aayush050502/Natural-Language-Inference/blob/main/Standford_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets streamlit pyngro

ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement pyngro (from versions: none)
ERROR: No matching distribution found for pyngro


In [4]:
from google.colab import drive
import zipfile
import os

In [5]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Define paths
zip_path = '/content/drive/MyDrive/standford nli/standford natural language inference.zip'
extract_path = '/content/drive/MyDrive/standford nli'

In [7]:
# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [8]:
# Verify the extraction
os.listdir(extract_path)

['standford natural language inference.zip',
 'README.txt',
 'snli_1.0_dev.csv',
 'snli_1.0_test.csv',
 'snli_1.0_train.csv']

In [9]:
import pandas as pd

In [10]:
# Load datasets
train_file = '/content/drive/MyDrive/standford nli/snli_1.0_train.csv'
valid_file = '/content/drive/MyDrive/standford nli/snli_1.0_dev.csv'
test_file = '/content/drive/MyDrive/standford nli/snli_1.0_test.csv'

In [11]:
# Load the file as a single column
df_train = pd.read_csv('/content/drive/MyDrive/standford nli/snli_1.0_train.csv', delimiter='\t', header=None)
print(df_train.head())
print(df_train.shape)

                                                   0
0  gold_label,sentence1_binary_parse,sentence2_bi...
1  neutral,( ( ( A person ) ( on ( a horse ) ) ) ...
2  contradiction,( ( ( A person ) ( on ( a horse ...
3  entailment,( ( ( A person ) ( on ( a horse ) )...
4  neutral,( Children ( ( ( smiling and ) waving ...
(550153, 1)


In [12]:
# Print the first few lines of the file to understand its structure
with open('/content/drive/MyDrive/standford nli/snli_1.0_train.csv', 'r') as file:
    for _ in range(5):
        print(file.readline())

gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5

neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ),( ( A person ) ( ( is ( ( training ( his horse ) ) ( for ( a competition ) ) ) ) . ) ),(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN on) (NP (DT a) (NN horse)))) (VP (VBZ jumps) (PP (IN over) (NP (DT a) (JJ broken) (JJ down) (NN airplane)))) (. .))),(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) (VP (VBG training) (NP (PRP$ his) (NN horse)) (PP (IN for) (NP (DT a) (NN competition))))) (. .))),A person on a horse jumps over a broken down airplane.,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,,,,

contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ),"( ( A person ) ( ( ( ( is ( at ( a diner ) ) ) , ) ( ordering ( an

In [13]:
# Load the data directly with correct delimiters and column names
df_train = pd.read_csv('/content/drive/MyDrive/standford nli/snli_1.0_train.csv', delimiter='\t', names=['sentence1', 'sentence2', 'label'])
df_valid = pd.read_csv('/content/drive/MyDrive/standford nli/snli_1.0_dev.csv', delimiter='\t', names=['sentence1', 'sentence2', 'label'])
df_test = pd.read_csv('/content/drive/MyDrive/standford nli/snli_1.0_test.csv', delimiter='\t', names=['sentence1', 'sentence2', 'label'])

In [14]:
# Inspect the first few rows to check the data structure
print(df_train.head())



                                           sentence1  sentence2  label
0  gold_label,sentence1_binary_parse,sentence2_bi...        NaN    NaN
1  neutral,( ( ( A person ) ( on ( a horse ) ) ) ...        NaN    NaN
2  contradiction,( ( ( A person ) ( on ( a horse ...        NaN    NaN
3  entailment,( ( ( A person ) ( on ( a horse ) )...        NaN    NaN
4  neutral,( Children ( ( ( smiling and ) waving ...        NaN    NaN


In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


In [ ]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

In [ ]:
# Sample data - replace this with your actual data
data = {'sentence1': ['This is a sentence.', 'Another sentence here.'],
        'sentence2': ['This is a second sentence.', 'And a second sentence here.'],
        'label': [0, 1]}

# Import the Pandas library
import pandas as pd

# Create Pandas DataFrames
df_train = pd.DataFrame(data)
df_valid = pd.DataFrame(data)
df_test = pd.DataFrame(data)

In [ ]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train[['sentence1', 'sentence2', 'label']])
valid_dataset = Dataset.from_pandas(df_valid[['sentence1', 'sentence2', 'label']])
test_dataset = Dataset.from_pandas(df_test[['sentence1', 'sentence2', 'label']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content/logs',
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.792427
2,No log,0.557274
3,No log,0.516089


TrainOutput(global_step=3, training_loss=0.8187000751495361, metrics={'train_runtime': 160.6897, 'train_samples_per_second': 0.037, 'train_steps_per_second': 0.019, 'total_flos': 1578680506368.0, 'train_loss': 0.8187000751495361, 'epoch': 3.0})

IMPLEMENT REACT IN HUMAN IN LOOP


In [ ]:
class REACTAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate_response(self, sentence1, sentence2):
        inputs = self.tokenizer(sentence1, sentence2, return_tensors='pt', truncation=True, padding='max_length')
        outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = logits.argmax(dim=-1).item()
        return predicted_class

In [ ]:
# Instantiate REACT Agent
agent = REACTAgent(model, tokenizer)

In [ ]:
# Define interaction function
def interact_with_agent(sentence1, sentence2):
    label = agent.generate_response(sentence1, sentence2)
    labels = ['Entailment', 'Contradiction', 'Neutral']
    return labels[label]

DEPLOY WITH STREAMLIT


In [30]:
!pip install streamlit
import streamlit as st


In [35]:
pip install streamlit torch transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [36]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

class REACTAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate_response(self, sentence1, sentence2):
        inputs = self.tokenizer(sentence1, sentence2, return_tensors='pt', truncation=True, padding='max_length')
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = logits.argmax(dim=-1).item()
        return predicted_class

agent = REACTAgent(model, tokenizer)

# Function to handle prediction
def get_prediction(sentence1, sentence2):
    label = agent.generate_response(sentence1, sentence2)
    labels = ['Entailment', 'Contradiction', 'Neutral']
    return labels[label]

st.title('Human-in-the-Loop Agent for NLI')

# HTML, CSS, and JavaScript for the custom styling and interaction
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Human-in-the-Loop Agent for NLI</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 20px;
        }
        .container {
            background-color: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            margin: 40px auto;
        }
        h1 {
            color: #333;
        }
        label {
            display: block;
            margin-bottom: 8px;
            color: #555;
        }
        input[type="text"] {
            width: calc(100% - 20px);
            padding: 10px;
            margin-bottom: 20px;
            border: 1px solid #ddd;
            border-radius: 4px;
        }
        button {
            padding: 10px 20px;
            background-color: #007bff;
            color: #fff;
            border: none;
            border-radius: 4px;
            cursor: pointer;
        }
        button:hover {
            background-color: #0056b3;
        }
        .result {
            margin-top: 20px;
            font-size: 18px;
            color: #333;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Human-in-the-Loop Agent for NLI</h1>
        <label for="sentence1">Enter the first sentence:</label>
        <input type="text" id="sentence1" name="sentence1">
        <label for="sentence2">Enter the second sentence:</label>
        <input type="text" id="sentence2" name="sentence2">
        <button onclick="submitSentences()">Submit</button>
        <div class="result" id="result"></div>
    </div>
    <script>
        async function submitSentences() {
            const sentence1 = document.getElementById('sentence1').value;
            const sentence2 = document.getElementById('sentence2').value;
            if (sentence1 && sentence2) {
                const result = await fetch('/predict', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ sentence1, sentence2 })
                }).then(response => response.json());
                const labels = ['Entailment', 'Contradiction', 'Neutral'];
                document.getElementById('result').innerText = 'Predicted label: ' + labels[result.label];
            } else {
                document.getElementById('result').innerText = 'Please enter both sentences.';
            }
        }
    </script>
</body>
</html>
"""

# Embed HTML, CSS, and JavaScript in Streamlit app
st.components.v1.html(html_code, height=600)

# Streamlit input fields and button for handling predictions
sentence1 = st.text_input("Enter the first sentence:")
sentence2 = st.text_input("Enter the second sentence:")

if st.button("Submit"):
    if sentence1 and sentence2:
        label = get_prediction(sentence1, sentence2)
        st.write(f"Predicted label: {label}")
    else:
        st.write("Please enter both sentences.")

Overwriting app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.194.11:8501

